In [5]:
import pandas as pd
from math import radians, cos, sin, sqrt, atan2

# 거리 계산 함수 (Haversine 공식)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0 # 지구의 반지름 (km)
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# 데이터 불러오기
df_districts = pd.read_excel('달서구 행정구역 중심좌표.xlsx')
transport_coords = pd.read_excel('지하철,버스 좌표.xlsx')

# 위도와 경도를 float 형식으로 변환
df_districts['위도'] = df_districts['위도'].astype(float)
df_districts['경도'] = df_districts['경도'].astype(float)
transport_coords['위도'] = transport_coords['위도'].astype(float)
transport_coords['경도'] = transport_coords['경도'].astype(float)

# 접근성 점수 계산 함수
def calculate_accessibility_score(lat, lon, transport_coords, radius=0.5):
    scores = []
    for _, row in transport_coords.iterrows():
        lat_t = row['위도']
        lon_t = row['경도']
        dist = haversine(lat, lon, lat_t, lon_t)
        if dist <= radius:
            score = 1 / (dist + 0.1)
            scores.append(score)
    return sum(scores)

# 각 행정구역에 대해 접근성 지수 계산
df_districts['접근성지수'] = df_districts.apply(
    lambda row: calculate_accessibility_score(row['위도'], row['경도'], transport_coords),
    axis=1
)

print(df_districts[['행정구역', '접근성지수']])


   행정구역      접근성지수
0   성당동  11.802993
1   두류동  26.359910
2   본리동  47.831097
3   감삼동  22.497932
4   죽전동  17.344520
5   장기동  22.638786
6   용산동  30.907863
7   이곡동  44.958868
8   신당동  30.365903
9   월성동  46.874559
10  진천동  33.029489
11  유천동  44.487457
12  상인동  34.032556
13  도원동   5.499351
14  송현동  26.987156
15   본동  35.625293


In [6]:
df_districts.to_excel('달서구 행정구역 접근성지수.xlsx', index=False)